In [1]:
!cp -r /kaggle/input/khrm-for-openpolymer /kaggle/working/.

In [2]:
!cp -r /kaggle/input/khrm-for-openpolymer /kaggle/working/.
!cd /kaggle/working/khrm-for-openpolymer
!pip install --no-cache-dir torch-geometric -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install --no-cache-dir torch-cluster -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install --no-cache-dir torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
#!pip install /kaggle/input/torch-geometric/torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl
!pip install  "pyscf>=2.10.0" "pyscf-semiempirical>=0.1.1" "pysmiles>=2.0.0" "rdkit>=2025.3.5"

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.0 MB/s eta 0:00:00a 0:00:01
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 123.7 MB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 32.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 45.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.1/126.1 kB 7.6 MB/s eta 0:00:00
  Created wheel for pyscf-semiempirical: filename=pyscf_semiempirical-0.1.1-cp311-cp311-linux_x86_64.whl size=57471 sha256=d72f0df6729da3962f7a37e286b71fb3b8830b4ca5a6

In [3]:
import os
import sys
import math
import json
from typing import List, Tuple, Dict
from datetime import datetime

import numpy as np
import pandas as pd

import torch

# Data props
PROPERTIES = ["Tg", "FFV", "Tc", "Density", "Rg"]
INPUTS = ["SMILES"]
AUX_INFO = ["monomer_count", "original_atoms", "final_atoms", "SMILES"]

# y output
TARGET_DIM = len(PROPERTIES)

# Params
EPOCHS = 10
BATCH_SIZE = 16
LR = 1e-3
WEIGHT_DECAY = 1e-4
CHECKPOINT_EVERY_N_STEPS = 100
K_HEADS = 8

# data params
VAL_RATIO = 0.1
MIN_LENGTH = 75
MAX_OUTPUT = 10


# Random seeding
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
import os

PROJECT_ROOT = "/kaggle/input/"
DATA_CSV = os.path.join(PROJECT_ROOT, "neurips-open-polymer-prediction-2025")
CHECKPOINT_DIR = os.path.join("/kaggle/working", "checkpoints", "hrm")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [5]:
# Load data

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

LOAD_COLS = (INPUTS + PROPERTIES)

raw_df = pd.DataFrame(columns=LOAD_COLS)
import os
for dirname, _, filenames in os.walk(PROJECT_ROOT + "/neurips-open-polymer-prediction-2025"):
    for filename in filenames:
        if ('dataset' in filename and '.csv' in filename) or 'train.csv' in filename:
            df_temp = pd.read_csv(os.path.join(dirname, filename))
            df_temp.reindex(columns=LOAD_COLS)
            raw_df = pd.concat([raw_df, df_temp], ignore_index=True)

print(len(raw_df))
raw_df = (
    raw_df
    .sort_values("SMILES")
    .groupby("SMILES", as_index=False)
    .first()
)
print(len(raw_df))

16963
10345


/tmp/ipykernel_36/2920036644.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  raw_df = pd.concat([raw_df, df_temp], ignore_index=True)


In [6]:
!pip install pysmiles
!pip install rdkit

In [7]:
import sys
sys.path.append("/kaggle/input/khrm-for-openpolymer/src")
import data_gen_helpers
from data_gen_helpers import iterative_extend_smiles, count_non_hydrogen_atoms

from IPython.display import clear_output
import logging
from datetime import datetime

from multiprocessing import Pool, cpu_count
N_PROCESSES = max(cpu_count()-1, 1)

def process_row(args):
    idx, row = args
    original_smiles = row["SMILES"]
    original_atoms = count_non_hydrogen_atoms(original_smiles)

    try:
        extensions = list(iterative_extend_smiles(
            original_smiles,
            min_length=MIN_LENGTH,
            max_output=MAX_OUTPUT
        ))

        results = []
        if extensions:
            for extended_smiles, monomer_count in extensions:
                final_atoms = count_non_hydrogen_atoms(extended_smiles)
                new_row = row.copy()
                new_row["SMILES"] = extended_smiles
                new_row["monomer_count"] = monomer_count
                new_row["original_smiles"] = original_smiles
                new_row["original_atoms"] = original_atoms
                new_row["final_atoms"] = final_atoms
                results.append(("success", new_row))
        else:
            return [("fail", (idx, original_smiles, "No extensions generated"))]

        # FIX: don't double-wrap
        return results

    except Exception as e:
        return [("fail", (idx, original_smiles, str(e)))]
        
# ---- Parallel Execution ----
def parallel_extend(raw_df):
    extended_data = []
    failed_extensions = []
    counter = 1
    with Pool(N_PROCESSES) as pool:
        for results in pool.imap_unordered(process_row, raw_df.iterrows(), chunksize=10):
            for status, payload in results:
                if status == "success":
                    extended_data.append(payload)
                else:
                    failed_extensions.append(payload)
            if counter % 100 == 0:
                clear_output(wait=True)  # clears the current cell output
                print(f"Reached mol {counter}/{len(raw_df)}")
                print(f"Successes: {len(extended_data)}")
                print(f"Failures: {len(failed_extensions)}")
            counter += 1

    # Build DataFrame
    mixed_fields=(["SMILES", "monomer_count", "original_smiles", "original_atoms", "final_atoms"]+PROPERTIES)
    extended_df = pd.DataFrame(extended_data, columns = mixed_fields)
    print(f"\nSuccessfully extended: {len(extended_df)} molecules")
    print(f"Failed extensions: {len(failed_extensions)}")

    if failed_extensions:
        print("\n\nFailed molecules:\n\n")
        for idx, smiles, error in failed_extensions[:5]:
            print(f"  {idx}: {smiles} - {error}")

    if len(extended_df) == 0:
        raise RuntimeError("No molecules could be extended. Check your data and extension logic.")

    return extended_df, failed_extensions

# ---- Run ----
extended_df, failed_extensions = parallel_extend(raw_df)
if len(failed_extensions) > 0:
    print("\n\nFailed molecules:\n\n")
    for idx, smiles, error in failed_extensions:  # Show first 5 errors
        print(f"  {idx}: {smiles} - {error}")

print(extended_df.columns)

# Use extended data for training
if len(extended_df) == 0:
    raise RuntimeError("No molecules could be extended. Check your data and extension logic.")


Reached mol 10300/10345
Successes: 59721
Failures: 162

Successfully extended: 59984 molecules
Failed extensions: 162


Failed molecules:


  70: */C=C(\C#N)C(=O)NC1CCCCC1NC(=O)/C(C#N)=C/c1ccc(/C=C/c2ccc(N(CC)Cc3cccc(CN(CC)c4ccc(/C=C/c5ccc(*)s5)cc4)c3)cc2)s1 - Dangling E/Z isomer token for double bond between 15 16.
  71: */C=C(\C#N)C(=O)NC1CCCCC1NC(=O)/C(C#N)=C/c1ccc(/C=C/c2ccc(N(CC)Cc3ccccc3CN(CC)c3ccc(/C=C/c4ccc(*)s4)cc3)cc2)s1 - Dangling E/Z isomer token for double bond between 15 16.
  72: */C=C(\C#N)C(=O)NC1CCCCC1NC(=O)/C(C#N)=C/c1ccc(/C=C/c2ccc(N(c3ccccc3)c3ccc(N(c4ccccc4)c4ccc(/C=C/c5ccc(*)s5)cc4)cc3)cc2)s1 - Dangling E/Z isomer token for double bond between 15 16.
  74: */C=C(\C#N)C(=O)Nc1cccc(NC(=O)/C(C#N)=C/c2ccc(/C=C/c3ccc(N(c4ccccc4)c4ccc(N(c5ccccc5)c5ccc(/C=C/c6ccc(*)s6)cc5)cc4)cc3)s2)c1 - Dangling E/Z isomer token for double bond between 14 15.
  75: */C=C(\C#N)C(=O)OCCCCCCCCCCOC(=O)/C(C#N)=C/c1ccc2c(c1)c1cc(*)ccc1n2CCCCCCC - Dangling E/Z isomer token for double bond bet

In [8]:
!pip install geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 6.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for geometric: filename=geometric-1.1-py3-none-any.whl size=402087 sha256=b1c403d712b42bd97a693e36d91c94ec55385c20bb13a2999353e648fad9e4e4
  Stored in directory: /root/.cache/pip/wheels/c5/77/0c/96a54539fe0560749fdbe283f92582d09bddb212856407a4cb
Successfully built geometric


In [9]:
from dataset_helpers import smiles_iter_to_graph_dataset

import numpy as np
from torch_geometric.loader import DataLoader

# Split into Train/Val
num_rows = len(extended_df)
perm = np.random.RandomState(SEED).permutation(num_rows)
train_count = int((1.0 - VAL_RATIO) * num_rows)
train_idx, val_idx = perm[:train_count], perm[train_count:]
train_df = extended_df.iloc[train_idx].reset_index(drop=True)
val_df = extended_df.iloc[val_idx].reset_index(drop=True)

# Create graph datasets
train_dataset = smiles_iter_to_graph_dataset(train_df["SMILES"], torch.tensor(train_df[PROPERTIES].to_numpy(), dtype=torch.float32, device=device), aux_info=train_df[AUX_INFO])
val_dataset = smiles_iter_to_graph_dataset(val_df["SMILES"], torch.tensor(val_df[PROPERTIES].to_numpy(), dtype=torch.float32, device=device), aux_info=val_df[AUX_INFO])

print(f"example aux_info: {train_dataset[0].aux_info}")
print(f"Train graphs: {len(train_dataset)} | Val graphs: {len(val_dataset)}")

# DataListLoader (batched lists of Data objects)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Derive input dimension from first training graph
if len(train_dataset) == 0:
    raise RuntimeError("Training dataset is empty after preprocessing.")
INPUT_DIM = train_dataset[0].x.shape[1]
print(f"Input dim: {INPUT_DIM}, Target dim: {TARGET_DIM}")

EDGE_DIM = train_dataset[0].edge_attr.shape[1]
print(f"Edge dim: {EDGE_DIM}")

# Show some examples of the extensions
print(f"\nExamples of SMILES extensions:")
for i in range(min(3, len(train_df))):
    row = train_df.iloc[i]
    print(f"Original ({row.get('original_atoms', 'N/A')} atoms): {row.get('original_smiles', 'N/A')}")
    print(f"Extended ({row.get('final_atoms', 'N/A')} atoms): {row['SMILES']}")
    print()

TOTAL_BATCH_COUNT = len(train_loader)

example aux_info: [4 11 46
 'CC(CCC(CC(CC(C)OC(=O)c1ccccc1)OC(=O)c1ccccc1)OC(=O)c1ccccc1)OC(=O)c1ccccc1']
Train graphs: 53985 | Val graphs: 5999
Input dim: 6, Target dim: 5
Edge dim: 4

Examples of SMILES extensions:
Original (11 atoms): *CC(*)OC(=O)c1ccccc1
Extended (46 atoms): CC(CCC(CC(CC(C)OC(=O)c1ccccc1)OC(=O)c1ccccc1)OC(=O)c1ccccc1)OC(=O)c1ccccc1

Original (59 atoms): *C(=O)Nc1ccc(C2(c3ccc(NC(=O)c4ccc5c(c4)C(=O)N(c4cccc(N6C(=O)c7ccc(*)cc7C6=O)c4)C5=O)cc3)c3ccccc3-c3ccccc32)cc1
Extended (120 atoms): Cc1ccc2c(c1)C(=O)N(c1cccc(N3C(=O)c4ccc(C(=O)Nc5ccc(C6(c7ccc(NC(=O)C(=O)Nc8ccc(C9(c%10ccc(NC(=O)c%11ccc%12c(c%11)C(=O)N(c%11cccc(N%13C(=O)c%14ccc(C)cc%14C%13=O)c%11)C%12=O)cc%10)c%10ccccc%10-c%10ccccc%109)cc8)cc7)c7ccccc7-c7ccccc76)cc5)cc4C3=O)c1)C2=O

Original (15 atoms): */C=C/[Si](*)(c1ccccc1)c1ccccc1
Extended (62 atoms): C[Si](C=C[Si](C=CC=C[Si](C=C[Si](C)(c1ccccc1)c1ccccc1)(c1ccccc1)c1ccccc1)(c1ccccc1)c1ccccc1)(c1ccccc1)c1ccccc1



In [10]:
import kmeans_hrm_model
# Use KMeansHRMModule from kmeans_hrm_model.py
from kmeans_hrm_model import (
    KMeansHRMModule, KMeansHRMConfig, KMeansHRMInnerModuleConfig, KMeansHRMInitialCarry,
    KMeansConfig, KMeansHeadConfig, OutputHeadConfig,
    SpectralWeighting, SpectralWeightingConfig,
    DiscreteMeanCenter, DiscreteMeanCenterConfig,
    RadiusAttentionWeights, RadiusMaskConfig
)
from torch.optim import Adam
from torch_geometric.nn import GATConv

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def create_kmeans_hrm_config(input_dim: int, edge_dim: int, hidden_dim: int = 512, latent_dim: int = 128, output_dim: int = TARGET_DIM, k: int = 8) -> KMeansHRMConfig:
    
    # Spectral Weighting Configuration
    spectral_config = SpectralWeightingConfig(
        node_channels=latent_dim,
        K=3,  # Chebyshev polynomial order
        num_layers=2,
        normalization='sym',
        bias=True,
        dropout=0.2,
        norm='batch',
        norm_kwargs={'in_channels': latent_dim}
    )
    
    # Center Module Configuration
    center_config = DiscreteMeanCenterConfig(
        distance_metric='euclidean'
    )
    
    # Radius Mask Configuration (simplified weighting module)
    radius_weighting = GATConv(latent_dim, latent_dim)
    radius_config = RadiusMaskConfig(
        max_num_neighbors=50,
        radius=20,
        weighting_module=radius_weighting,
        threshold=0.1,
        node_dim=latent_dim
    )
    
    # KMeans Head Configuration
    kmeans_head_config = KMeansHeadConfig(
        node_count=k,
        node_dim=latent_dim,
        max_nodes=80,  
        num_layers=5,
        dropout=0.2,
        weighting_module=SpectralWeighting(spectral_config),
        center_module=DiscreteMeanCenter(center_config),
        mask_module=RadiusAttentionWeights(radius_config),
        act='relu',
        act_kwargs={}
    )
    
    # KMeans Configuration
    kmeans_config = KMeansConfig(
        k=k,
        max_iter=15,
        thresh=1e-6,
        max_overlap=2,
        head_module=kmeans_head_config,
        excluded_is_cluster=True
    )
    
    # Output Head Configuration
    output_head_config = OutputHeadConfig(
        node_dim=latent_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        pooling_type='mean',
        norm='batch',
        norm_kwargs={'in_channels': hidden_dim},
        act='relu',
        act_kwargs={}
    )
    
    # Policy Module Configuration
    policy_config = OutputHeadConfig(
        node_dim=latent_dim,
        hidden_dim=hidden_dim//2,
        output_dim=2,  # halt=0, continue=1
        pooling_type='mean',
        norm='batch',
        norm_kwargs={'in_channels': hidden_dim//2},
        act='relu',
        act_kwargs={}
    )
    
    # Inner Module Configuration
    inner_config = KMeansHRMInnerModuleConfig(
        add_self_loops=True,
        add_negative_edges=True,
        dropout=0.2,
        hidden_dim=hidden_dim,        # inner-side hidden size, reused
        node_dim=latent_dim,          # must equal vgae_latent_dim
        attention_dim=4,           # bigger in final
        edge_dim=edge_dim,
        layers=3,
        kmeans_config=kmeans_config,
        output_head_config=output_head_config,
        policy_module_config=policy_config,
        K_cycles=3,
        L_cycles=5,
        batch_size=BATCH_SIZE,
        halt_max_steps=10,
        halt_exploration_prob=0.1,
    )
    
    config = KMeansHRMConfig(
        inner_module=inner_config,
        explore_steps_prob=0.1,
        halt_max_steps=10,
        pre_encoder_conv_layers=2,
        vgae_encoder_type="cheb",
        input_dim=input_dim,
        edge_attr_dim=edge_dim,
        vgae_latent_dim=latent_dim,           # must equal inner.node_dim
        vgae_encoder_layers=2,
        vgae_encoder_dropout=0.1,
        vgae_decoder_type=None,
        vgae_kl_weight=1.0,
    )
    
    return config

# Modell initialisieren
hrm_config = create_kmeans_hrm_config(INPUT_DIM, EDGE_DIM, latent_dim=32, hidden_dim=256, output_dim=TARGET_DIM, k=8)
#hrm_config = create_kmeans_hrm_config(INPUT_DIM, EDGE_DIM, latent_dim=4, hidden_dim=4, output_dim=TARGET_DIM, k=2)
model = KMeansHRMModule(hrm_config, training=True).to(device)
optimizer = Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# Print number of parameters
num_params = count_parameters(model)
print(f"KMeansHRMModule created with {num_params:,} trainable parameters")
print(f"Model size: {num_params * 4 / 1024 / 1024:.2f} MB (float32)")
print(model)


KMeansHRMModule created with 71,884 trainable parameters
Model size: 0.27 MB (float32)
KMeansHRMModule(
  (inner_module): KMeansHRMInnerModule(
    (kmeans_module): KMeans(
      (heads): ModuleList(
        (0-7): 8 x KMeansHead(
          (weighting_module): SpectralWeighting(
            (cheb_convs): ModuleList(
              (0-1): 2 x ChebConv(32, 32, K=3, normalization=sym)
            )
            (norms): ModuleList(
              (0-1): 2 x BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
          (center_module): DiscreteMeanCenter(
            (distance_module): PairwiseDistance()
          )
          (mask_module): RadiusAttentionWeights(
            (weighting_module): GATConv(32, 32, heads=1)
            (_mask_linear): Linear(in_features=32, out_features=1, bias=True)
          )
          (act): ReLU()
        )
      )
    )
    (vgae_encoder): VGAEEncoder(
      (norms): ModuleList(
        (0-2): 3 x LayerNor

In [11]:
import training_loops
from training_loops import composite_loss, compute_mae_in_bounds, init_property_bounds

init_property_bounds(PROPERTIES, extended_df)

from typing import TypedDict, Dict, Any
from dataclasses import dataclass

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split
from torch.optim import Adam

from torch_geometric.data import Data, Batch

def save_checkpoint(state: Dict, step: int, is_best: bool = False):
    path = os.path.join(CHECKPOINT_DIR, f"step_{step}.pt")
    torch.save(state, path)
    if is_best:
        best_path = os.path.join(CHECKPOINT_DIR, "best.pt")
        torch.save(state, best_path)


class PretrainConfig(TypedDict):
    # Data
    data_path: str

    # Hyperparams
    global_batch_size: int
    epochs: int
    total_iters: int

    lr: float
    lr_min_ratio: float
    lr_warmup_steps: int

    weight_decay: float
    beta1: float
    beta2: float


@dataclass
class TrainState:
    model: nn.Module
    optimizer: torch.optim.Optimizer
    carry: KMeansHRMInitialCarry | None

    step: int
    total_steps: int


def compute_warmup_weight(step: int, warmup_steps: int, min_ratio: float) -> float:
    if warmup_steps <= 0:
        return 1.0
    if step < warmup_steps:
        # Linear warmup from min_ratio -> 1.0
        return float(min_ratio + (1.0 - min_ratio) * (step / max(1, warmup_steps)))
    return 1.0


def pack_train_state_for_save(ts: TrainState) -> Dict[str, Any]:
    return {
        "step": int(ts.step),
        "total_steps": int(ts.total_steps),
        # Carry can be large; still useful for exact resume within the same batch sequence
        "carry": ts.carry,
    }


Loss and metrics initialized.


In [12]:
from typing import Union, List
from training_loops import range_violation_loss

def composite_loss(monomer_index: int, properties: List[str], preds: torch.Tensor, targets: torch.Tensor, related_info: torch.Tensor) -> torch.Tensor:
    # preds: [B, 5], targets: [B, 5] with NaNs if missing
    loss_items = []
    
    related_info.unsqueeze(-1)
    monomer_counts = torch.tensor(related_info[:, monomer_index], dtype=torch.float32, device=preds.device)
    weight = torch.pow(0.3, monomer_counts - 1).to(preds.device).view(-1)

    for j in range(len(properties)):
        t = targets[:, j]
        p = preds[:, j]
        mask_present = torch.isfinite(t).to(p.device)

        assert p.device == t.device, f"p.device: {p.device}, t.device: {t.device}"
        assert p.device == weight.device, f"p.device: {p.device}, related_info.device: {weight.device}"
        assert mask_present.shape == p.shape, f"mask_present.shape: {mask_present.shape}, p.shape: {p.shape}"
        assert mask_present.shape == t.shape, f"mask_present.shape: {mask_present.shape}, t.shape: {t.shape}"
        assert mask_present.shape == weight.shape, f"mask_present.shape: {mask_present.shape}, weight.shape: {weight.shape}"


        if mask_present.any():
            mse = F.mse_loss(p[mask_present], t[mask_present], weight=weight[mask_present])
            loss_items.append(mse)
        # Range-violation for all (including available): no loss if within bounds
        rv = range_violation_loss(properties, p, j).mean()
        loss_items.append(rv)
    return torch.stack(loss_items).mean()


@torch.no_grad()
def compute_mae_in_bounds(monomer_index: int, properties: List[str], preds: torch.Tensor, targets: torch.Tensor, related_info: torch.Tensor) -> Dict[str, float]:
    out = {}

    assert type(related_info) == torch.Tensor, f"related info is a {type(related_info)}"

    monomer_counts = torch.tensor(related_info[:, monomer_index], dtype=torch.float32, device=preds.device)
    weight = torch.pow(0.3, monomer_counts - 1).to(preds.device).view(-1)

    for j, name in enumerate(properties):

        t = targets[:, j]
        p = preds[:, j]
        mask_present = torch.isfinite(t).to(p.device)

        if mask_present.any():
            out[f"mae_{name}"] = (torch.mul(p[mask_present] - t[mask_present], weight[mask_present])).abs().mean().item()
        else:
            out[f"mae_{name}"] = torch.tensor(float('nan'))
    return out

print("Loss and metrics initialized.")

Loss and metrics initialized.


In [15]:
def train_batch(epoch: int, train_state: TrainState, batch_data: Batch, config: PretrainConfig) -> Dict[str, float]:
    model = train_state.model
    optimizer = train_state.optimizer

    model.train()

    # Targets [B,5]
    y = torch.stack([g.y for g in batch_data.to_data_list()], dim=0).to(device)
    batch_data = batch_data.to(device)

    related_info = torch.stack([torch.as_tensor(d[0]).to(device) for d in batch_data.aux_info]).unsqueeze(1).to(device)
    #related_info = batch_data.aux_info
    
    # Reuse existing carry if provided; otherwise initialize or realign to current num_graphs
    need_new_carry = (
        train_state.carry is None
        or (hasattr(batch_data, 'num_graphs') and train_state.carry.steps.numel() != batch_data.num_graphs)
    )
    if need_new_carry:
        train_state.carry = model.initial_carry(batch_data)

    # Forward
    train_state.carry, hrm_output = model(train_state.carry, batch_data)
    preds = hrm_output['y_pred'].to(device)
    policy = hrm_output['q_policy']
    target_policy = hrm_output.get('target_q_policy', None)

    targets = y
    properties = PROPERTIES

    if target_policy is not None and target_policy[0].size == policy[0].size:

        preds = torch.cat([preds, policy[0].unsqueeze(-1), policy[1].unsqueeze(-1)], dim=1)
        targets = torch.cat([y, target_policy[0].unsqueeze(-1), target_policy[1].unsqueeze(-1)], dim=1)

        properties += ['q_halt', 'q_cont']

    assert preds.size(0) == targets.size(0), f"batch mismatch: preds {preds.shape}, targets {targets.shape}"
    assert len(properties) == targets.size(1), f"mismatch {y.size} {properties}"

    loss = composite_loss(0, properties, preds, targets, related_info)

    train_state.step += 1
    warmup_w = compute_warmup_weight(train_state.step, config["lr_warmup_steps"], config["lr_min_ratio"])  # type: ignore[index]
    scaled_loss = loss * warmup_w * (1.0 / TOTAL_BATCH_COUNT)

    # Backward/update
    optimizer.zero_grad()
    scaled_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
    optimizer.step()

    # Metrics (unscaled loss for logging)
    with torch.no_grad():
        metrics = compute_mae_in_bounds(0, properties, preds, targets, related_info)
        metrics.update({
            "loss": loss.item(),
            "warmup_weight": float(warmup_w),
        })


    # Periodic checkpoint
    global_step = train_state.step
    if global_step % CHECKPOINT_EVERY_N_STEPS == 0:
        save_checkpoint({
            "epoch": epoch,
            "global_step": global_step,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "model_class": model.__class__.__name__,
            "model_module": model.__class__.__module__,
            "train_state": pack_train_state_for_save(train_state),
        }, step=global_step)

    return metrics


@torch.no_grad()
def validate(epoch: int, model: nn.Module, loader: DataLoader) -> Dict[str, float]:
    model.eval()
    running_loss = 0.0
    mae_accum = {f"mae_{p}": 0.0 for p in PROPERTIES + ['q_halt', 'q_cont']}
    count_samples = 0

    for batch_data in loader:
        # Targets [B,5]
        y = torch.stack([g.y for g in batch_data.to_data_list()], dim=0).to(device)
        batch_data = batch_data.to(device)

        related_info = torch.stack([torch.as_tensor(d[0]).to(device) for d in batch_data.aux_info]).unsqueeze(1).to(device)
        #related_info = batch_data.aux_info
        
        carry = model.initial_carry(batch_data)
        
        carry, hrm_output = model(carry, batch_data)
        preds = hrm_output['y_pred'].to(device)
        policy = hrm_output['q_policy']
        target_policy = hrm_output.get('target_q_policy', None)
        
        targets = y
        properties = PROPERTIES

        if target_policy is not None and target_policy[0].size == policy[0].size:
    
            preds = torch.cat([preds, policy[0].unsqueeze(-1), policy[1].unsqueeze(-1)], dim=1)
            targets = torch.cat([y, target_policy[0].unsqueeze(-1), target_policy[1].unsqueeze(-1)], dim=1)
    
            properties += ['q_halt', 'q_cont']

        assert preds.size(0) == y.size(0), f"batch mismatch: preds {preds.shape}, targets {targets.shape}"
        assert len(properties) == targets.size(1), f"mismatch {y.size} {properties}"

        loss = composite_loss(0, properties, preds, targets, related_info)

        metrics = compute_mae_in_bounds(0, properties, preds, targets, related_info)
        for k, v in metrics.items():
            if not math.isnan(v):
                mae_accum[k] += v * preds.size(0)
        running_loss += loss.item() * preds.size(0)
        count_samples += preds.size(0)

    avg_loss = running_loss / max(1, count_samples)
    avg_mae = {k: (v / max(1, count_samples)) for k, v in mae_accum.items()}

    return {"loss": avg_loss, **avg_mae}


In [16]:
# Added after failure at 3375 due to misplaced .cpu, changed from 0.001 to 0.01
optimizer = Adam(model.parameters(), lr=0.01, weight_decay=WEIGHT_DECAY)

In [17]:
from training_loops import train_until_total_iters

# Build a minimal config for warmup from existing hyperparams
CONFIG: PretrainConfig = {
    "data_path": "",
    "global_batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    # Step-based training support; used by helper loop
    "total_iters": int(EPOCHS * len(train_loader)),
    "lr": 0.01,
    "lr_min_ratio": 0.1,
    "lr_warmup_steps": max(1, len(train_loader) * 2),  # warm up first ~2 epochs of steps
    "weight_decay": WEIGHT_DECAY,
    "beta1": 0.9,
    "beta2": 0.999,
}

history = {"train": [], "val": []}
best_val_loss = float("inf")

# Initialize TrainState with total steps estimated from loader length and epochs
train_state = TrainState(
    model=model,
    optimizer=optimizer,
    carry=None,
    step=3375,
    total_steps=0  #EPOCHS * len(train_loader)
)

last_completed_epoch=0

history, best_val_loss = train_until_total_iters(
    total_iters=CONFIG["total_iters"],
    start_epoch=last_completed_epoch,
    train_state=train_state,
    train_loader=train_loader,
    val_loader=val_loader,
    config=CONFIG,
    train_batch_fn=train_batch,
    validate_fn=validate,
    save_checkpoint_fn=save_checkpoint,
    checkpoint_every_n=10,
    print_every_n=1
)

metrics_path = os.path.join(CHECKPOINT_DIR, "history.json")
with open(metrics_path, "w") as f:
    json.dump(history, f, indent=2)
print(f"Saved metrics to {metrics_path}")

step   3376/33750 | epoch   1 | batch    1 | loss=6673.9292 | lr=0.01
step   3377/33750 | epoch   1 | batch    2 | loss=87.2383 | lr=0.01
step   3378/33750 | epoch   1 | batch    3 | loss=0.6429 | lr=0.01
step   3379/33750 | epoch   1 | batch    4 | loss=0.0364 | lr=0.01
step   3380/33750 | epoch   1 | batch    5 | loss=547.2436 | lr=0.01
step   3381/33750 | epoch   1 | batch    6 | loss=432.6966 | lr=0.01
step   3382/33750 | epoch   1 | batch    7 | loss=5.5850 | lr=0.01
step   3383/33750 | epoch   1 | batch    8 | loss=581.3133 | lr=0.01
step   3384/33750 | epoch   1 | batch    9 | loss=11.8579 | lr=0.01
step   3385/33750 | epoch   1 | batch   10 | loss=20.0168 | lr=0.01
step   3386/33750 | epoch   1 | batch   11 | loss=885.5467 | lr=0.01
step   3387/33750 | epoch   1 | batch   12 | loss=3460.3269 | lr=0.01
step   3388/33750 | epoch   1 | batch   13 | loss=1211.5056 | lr=0.01
step   3389/33750 | epoch   1 | batch   14 | loss=0.0066 | lr=0.01
step   3390/33750 | epoch   1 | batch   15

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


metrics_path = os.path.join(CHECKPOINT_DIR, "history.json")
with open(metrics_path, "w") as f:
    json.dump(history, f, indent=2)
print(f"Saved metrics to {metrics_path}")

# Plot
def plot_curves(history):
    epochs = [e["epoch"] for e in history["train"]]
    train_losses = [e["loss"] for e in history["train"]]
    val_losses = [e["loss"] for e in history["val"]]

    plt.figure(figsize=(8,4))
    plt.plot(epochs, train_losses, label="train_loss")
    plt.plot(epochs, val_losses, label="val_loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_curves(history)

In [21]:
from dataset_helpers import smiles_to_graph_data
import pandas as pd
from torch_geometric.loader import DataLoader

df = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')

COLS = (["SMILES", "id"] + PROPERTIES)

submission_df = pd.DataFrame(columns=COLS)
submission_df = pd.concat([submission_df, df], ignore_index=True)

inputs = df["SMILES"].tolist()
id = df["id"].tolist()

graph_data_list = [smiles_to_graph_data(smiles, None, [info]) for smiles, info in zip(inputs, id)]
print(graph_data_list)

test_loader = DataLoader(graph_data_list, batch_size=1, shuffle=False)

submission_df

[Data(x=[39, 6], edge_index=[2, 84], edge_attr=[84, 4], aux_info=[1]), Data(x=[39, 6], edge_index=[2, 86], edge_attr=[86, 4], aux_info=[1]), Data(x=[44, 6], edge_index=[2, 98], edge_attr=[98, 4], aux_info=[1])]


,SMILES,id,Tg,FFV,Tc,Density,Rg
0,*Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)...,1109053969,NaN,NaN,NaN,NaN,NaN
1,*Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c...,1422188626,NaN,NaN,NaN,NaN,NaN
2,*c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6...,2032016830,NaN,NaN,NaN,NaN,NaN


In [38]:
model.eval()

max_iters = 10


with torch.no_grad():
    for idx, datum in enumerate(test_loader):
        preds = [0, 0, 0, 0, 0]
        carry = model.initial_carry(datum, device)
        while max_iters > 0:
            carry, hrm_output = model(carry, datum)

            preds = hrm_output['y_pred']
            q_policy = hrm_output['q_policy']

            if q_policy[0] > q_policy[1]:
                break
            max_iters -= 1
        max_iters = 10
        print(preds.cpu().numpy(), i)

        submission_df.loc[idx, ["Tg", "FFV", "Tc", "Density", "Rg"]] = preds.cpu().numpy().ravel()
        
print('submission_df', submission_df)

[[ 3.0364149e+01  2.7716428e-01 -2.2286460e-02  7.6941609e-01
   1.2645821e+01]] 2
[[ 2.7746019e+01  2.7507359e-01 -8.0580926e-03  7.6476061e-01
   1.2556776e+01]] 2
[[20.416775    0.2680679   0.03186549  0.7492246  12.277412  ]] 2
submission_df                                               SMILES          id         Tg  \
0  *Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)...  1109053969  30.364149   
1  *Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c...  1422188626  27.746019   
2  *c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6...  2032016830  20.416775   

        FFV        Tc   Density         Rg  
0  0.277164 -0.022286  0.769416  12.645821  
1  0.275074 -0.008058  0.764761  12.556776  
2  0.268068  0.031865  0.749225  12.277412  


In [39]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)